In [1]:
# import tools
from dotenv import load_dotenv
load_dotenv()
import os
import pytesseract 
from pdf2image import convert_from_path

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large", api_key=openai_api_key)

In [4]:
llm = ChatOpenAI(model="gpt-4o", api_key=openai_api_key)

### Extract data from pdf

In [126]:
# setup tesseract path for windows
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [5]:
file_path = "../artifacts/data/HSC26-Bangla1st-Paper.pdf"
text_file_path = "../artifacts/data/full_text_path.txt"

In [128]:
# # load documents
# docs = ""
# images = convert_from_path(file_path, dpi=300)
# for i, img in enumerate(images):
#     text = pytesseract.image_to_string(img, lang="ben")
#     docs += text


# with open(text_file_path, 'w', encoding="utf-8") as file:
#     file.write(docs)

### Load text

In [6]:
loader = TextLoader(text_file_path)
docs = loader.load()

In [7]:
# split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': '../artifacts/data/full_text_path.txt'}, page_content='========== Page 1 ==========\n10501০০\n\nঅনলাইন ব্যাচ"\n\nত\nবাংলা * ইংরেজি * আইসিটি\n\nঅনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,\n\nকল করো ত 16910\n\n========== Page 2 ==========\n115026\nঅনলাইন ব্যাচ কাটল\n] ১২০০\n\n/ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।\n/ তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।\n/ তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।\n\n/ তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে।\n/ নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে\nপারবে।\n/ মানুষ আশা নিয়ে বেঁচে থাকে- অনুপমের দৃষ্টান্তে মানবজীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে\nজ্ঞানলাভ করবে।\nছ প্রাক-মূল্যায়ন\n অনুপমের বাবা ওকালতি করে জীবিকা নির্বাহ করতেন।\n মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার- বিচক্ষণতা।\n\nনিচের অনুচ্ছেদটি পড়ে ৩ ও ৪ সংখ্যক প্রশ্নের উত্তর দাও।\n\nপিতূহীন দীপুর 

In [8]:
len(documents)

41

In [ ]:
# save vector store database
vector_store = FAISS.from_documents(documents, embedding)
vector_store.save_local("vectorstore")

In [8]:
# question prompt
question_prompt = ChatPromptTemplate.from_template("""
You are an AI assistant that generates meaningful questions from a given text.
Your task is to generate unique questions.
Context:
{text}

Instructions:
1. Read the text carefully.
2. Generate  unique questions clear, concise, and relevant questions based on the text.
3. Make sure the questions are varied: some factual, some analytical, and some conceptual.
4. Avoid yes/no questions.
5. Output the questions as a numbered list.
6. Do not translate into English or any other language; only Bangla.

Questions:
""")

In [9]:
refine_prompt = ChatPromptTemplate.from_template("""
You are an expert AI editor specialized in educational content. Your task is to refine or improve the given questions based on the additional context provided.
**only Bangla**
                                                 
Existing Questions:
{existing_questions}

Additional Context:
{input}

Instructions:
1. Carefully review the existing questions and the context.
2. Refine the questions to make them:
   - Clear, concise, and grammatically correct
   - Engaging, thought-provoking, and relevant to the context
3. If necessary, you may add 1-2 new questions inspired by the context.
4. Maintain the original intent of the existing questions.
5. If the additional context is not helpful, retain the original questions without change.
6. Do not translate into English or any other language; only Bangla.

Refined Questions:
""")

In [10]:
question_prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nYou are an AI assistant that generates meaningful questions from a given text.\nYour task is to generate unique questions.\nContext:\n{text}\n\nInstructions:\n1. Read the text carefully.\n2. Generate  unique questions clear, concise, and relevant questions based on the text.\n3. Make sure the questions are varied: some factual, some analytical, and some conceptual.\n4. Avoid yes/no questions.\n5. Output the questions as a numbered list.\n6. Do not translate into English or any other language; only Bangla.\n\nQuestions:\n'), additional_kwargs={})])

In [11]:
parser = StrOutputParser()

In [12]:
question_chain = question_prompt | llm | parser

In [13]:
refine_chain = refine_prompt | llm | parser

In [14]:
all_questions = []
for document in documents:
    # Generate initial questions
    initial_questions = question_chain.invoke({"text": document.page_content})
    
    # Refine questions using the same context
    refined_questions = refine_chain.invoke({
        "existing_questions": initial_questions,
        "input": document.page_content
    })
    
    # Parse the refined questions
    questions = [q.strip() for q in refined_questions.split("\n") if q.strip() != ""]
    
    for q in questions:
        if q not in all_questions:
            all_questions.append(q)

    if len(all_questions) > 20:
        break

print("Final refined questions:")
all_questions

Final refined questions:


['1. অনলাইন ব্যাচের সঙ্গে সম্পর্কিত যেকোনো জিজ্ঞাসার উত্তর পেতে কী করা উচিত?',
 '2. সমাজে হঠাৎ বিত্তশালী হয়ে ওঠা নিম্নবিত্ত ব্যক্তির কারণে কী ধরনের পরিচয় সংকট দেখা দিতে পারে?',
 '3. তৎকালীন সমাজে পণপ্রথার কুপ্রভাব সম্পর্কে আপনার কী ধারণা?',
 '4. "নারী কোমল হলেও দুর্বল নয়" - এই সত্য কল্যাণীর জীবনগল্পে কীভাবে প্রতিভাত হয়েছে?',
 '5. অনুপমের দৃষ্টান্তে মানবজীবনের চিরন্তন সত্য, আশার গুরুত্ব কীভাবে ফুটে উঠেছে?',
 "6. দীপু ও তার চাচার সম্পর্কের মধ্যে 'অপরিচিতা' গল্পের মামার চরিত্রের সঙ্গে কী ধরনের সাদৃশ্য আছে? দয়া করে উদাহরণসহ ব্যাখ্যা করুন।",
 '7. অনুপমের আত্মসমালোচনার মাধ্যমে তার জীবনের কোন দিকগুলো তুচ্ছ মনে হয় এবং কেন?',
 '8. দেবী দুর্গার দুই পুত্রের নাম কী এবং তাদের মধ্যে কে বড় এবং কে ছোট?',
 '9. ভারতের গয়া অঞ্চলের অন্তঃসলিলা নদীর কোন বিশেষ বৈশিষ্ট্য আছে?',
 '10. অনুপমের মামার চরিত্র-বৈশিষ্ট্য সম্পর্কে উপমা দ্বারা কী বোঝানো হয়েছে?',
 'নতুন প্রশ্ন:',
 '11. দীপু কীভাবে তার চাচার কর্তৃত্ব মেনে নেয় এবং তার জীবনে এর প্রভাব কী?',
 '1. অনুপম কেন তার বিবাহযাত্রার পরিস্থিতিকে সংগীত সরস্ব

In [ ]:
# all_question = []
# for document in documents:
#     question = question_chain.invoke({"text": document.page_content})
#     questions = [q.strip() for q in question.split("\n") if q.strip() != ""]
#     for q in questions:
#         if q not in all_question:
#             all_question.append(q)

#     if len(all_question) > 20:
#         break


# all_question

['1. অনলাইন ব্যাচ সম্পর্কিত যেকোনো তথ্য জানার জন্য কোন নম্বরে কল করা উচিত?',
 '2. সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানার জন্য কোন প্রসঙ্গের উপর আলোচনা হয়েছে?',
 '3. তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে কী আলোচনা করা হয়েছে?',
 "4. 'অপরিচিতা' গল্পের মামা চরিত্রের সঙ্গে দীপুর চাচার মিল কোন দিকগুলোতে পাওয়া যায়?",
 '5. অনুপমের জীবনের তুচ্ছতা প্রসঙ্গে তার আত্মসমালোচনার কোন বিষয়গুলো উল্লেখ করা হয়েছে?',
 '6. গুটি এবং সিজের নিরর্থক জীবনের তুলনা করে অনুপম কী বোঝাতে চেয়েছেন?',
 '7. অনুপম তার মামার চরিত্র-বৈশিষ্ট্য প্রসঙ্গে কীভাবে সংসারের দায়িত্ব পালন সম্পর্কে উপমা ব্যবহার করেছেন?',
 '8. যৌতুকের কারণে বিয়ের আলোচনা ভেঙে যাওয়া বিষয়ে দীপুর কী প্রতিক্রিয়া ছিল?',
 '9. দেবী দুর্গার পুত্ররা কারা এবং তাদের মধ্যে সম্পর্ক কেমন?',
 '10. অনুপমের মামার বিচক্ষণতায় তাকে ভাগ্য দেবতার প্রধান এজেন্ট বলা হয়েছে - এর পেছনের কারণ কী হতে পারে?',
 '১. অনুপম তার মামার চরিত্র-বৈশিষ্ট্য কিভাবে বর্ণনা করেছে?',
 '২. সংসারের দায়-দায়িত্ব পালনে অনুপমের ভূমিকা কিভাবে উপমার মাধ্যমে ব্যক্ত হয়েছে?',
 '৩. 

In [15]:
len(all_questions)

25

In [16]:
for q in all_questions:
    print(q)

1. অনলাইন ব্যাচের সঙ্গে সম্পর্কিত যেকোনো জিজ্ঞাসার উত্তর পেতে কী করা উচিত?
2. সমাজে হঠাৎ বিত্তশালী হয়ে ওঠা নিম্নবিত্ত ব্যক্তির কারণে কী ধরনের পরিচয় সংকট দেখা দিতে পারে?
3. তৎকালীন সমাজে পণপ্রথার কুপ্রভাব সম্পর্কে আপনার কী ধারণা?
4. "নারী কোমল হলেও দুর্বল নয়" - এই সত্য কল্যাণীর জীবনগল্পে কীভাবে প্রতিভাত হয়েছে?
5. অনুপমের দৃষ্টান্তে মানবজীবনের চিরন্তন সত্য, আশার গুরুত্ব কীভাবে ফুটে উঠেছে?
6. দীপু ও তার চাচার সম্পর্কের মধ্যে 'অপরিচিতা' গল্পের মামার চরিত্রের সঙ্গে কী ধরনের সাদৃশ্য আছে? দয়া করে উদাহরণসহ ব্যাখ্যা করুন।
7. অনুপমের আত্মসমালোচনার মাধ্যমে তার জীবনের কোন দিকগুলো তুচ্ছ মনে হয় এবং কেন?
8. দেবী দুর্গার দুই পুত্রের নাম কী এবং তাদের মধ্যে কে বড় এবং কে ছোট?
9. ভারতের গয়া অঞ্চলের অন্তঃসলিলা নদীর কোন বিশেষ বৈশিষ্ট্য আছে?
10. অনুপমের মামার চরিত্র-বৈশিষ্ট্য সম্পর্কে উপমা দ্বারা কী বোঝানো হয়েছে?
নতুন প্রশ্ন:
11. দীপু কীভাবে তার চাচার কর্তৃত্ব মেনে নেয় এবং তার জীবনে এর প্রভাব কী?
1. অনুপম কেন তার বিবাহযাত্রার পরিস্থিতিকে সংগীত সরস্বতীর পদ্মবন দলিত হওয়ার ঘটনার সঙ্গে তুলনা করেছে, এব

In [17]:
answer_prompt = ChatPromptTemplate.from_template("""
You are an AI assistant that provides accurate and concise answers to the given question on the context below.
Use only the information retrieved by the system to answer.
<context>
{context}
</context>                                                                                     
Question: {input}
Instructions:
1. Provide a clear and concise answer to the question.
2. Answer must be in Bangla.
3. Do not add extra commentary or explanations; only answer the question.
4. If the answer is not in the context, respond with "উত্তর উপলব্ধ নয়।"

Answer:
""")

In [18]:
load_vector_store = FAISS.load_local('vectorstore', embeddings=embedding, allow_dangerous_deserialization=True)
retriever = load_vector_store.as_retriever()


In [19]:
answer_chain = create_stuff_documents_chain(llm=llm, prompt=answer_prompt)
rag_chain = create_retrieval_chain(retriever, answer_chain)

In [20]:
answer_prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nYou are an AI assistant that provides accurate and concise answers to the given question on the context below.\nUse only the information retrieved by the system to answer.\n<context>\n{context}\n</context>                                                                                     \nQuestion: {input}\nInstructions:\n1. Provide a clear and concise answer to the question.\n2. Answer must be in Bangla.\n3. Do not add extra commentary or explanations; only answer the question.\n4. If the answer is not in the context, respond with "উত্তর উপলব্ধ নয়।"\n\nAnswer:\n'), additional_kwargs={})])

In [21]:
res = rag_chain.invoke({"input": "অনুপমের জীবন সম্পর্কে গল্পে কোন মন্তব্য করা হয়েছে"})
res['answer']

'গল্পে উল্লেখ করা হয়েছে যে অনুপম তার জীবনে আত্মসমালোচনা করেছে এবং পরিমাণ ও গুণ উভয় দিক দিয়েই তার জীবনটি তুচ্ছ বলে ব্যক্ত করেছে। অনুপম একটি ব্যক্তি্ত্বহীন যুবক, যার বিরহের জন্য সে নিজেই দায়ী, কারণ তার বিরহের মূল কারণ তার অক্ষমতা।'

In [22]:
for question in all_questions:
    print("Question ==> ", question)
    response = rag_chain.invoke({"input": question})
    print(response['answer'])

    with open("question_answer.txt", 'a', encoding='utf-8') as f:
        f.write("Question: "+ question + '\n')
        f.write("Answer: "+ response['answer'] + '\n\n')
        

Question ==>  1. অনলাইন ব্যাচের সঙ্গে সম্পর্কিত যেকোনো জিজ্ঞাসার উত্তর পেতে কী করা উচিত?
উত্তর উপলব্ধ নয়।
Question ==>  2. সমাজে হঠাৎ বিত্তশালী হয়ে ওঠা নিম্নবিত্ত ব্যক্তির কারণে কী ধরনের পরিচয় সংকট দেখা দিতে পারে?
উত্তর উপলব্ধ নয়।
Question ==>  3. তৎকালীন সমাজে পণপ্রথার কুপ্রভাব সম্পর্কে আপনার কী ধারণা?
তৎকালীন সমাজে পণপ্রথার কুপ্রভাব সম্পর্কে ধারণা পাওয়া যায় যে, এই প্রথার কারণে বরের বাবা যৌতুকের জন্য চাহিদা পূরণের ক্ষেত্রে কন্যার পিতাকে প্রায়শই অপমানিত ও সর্বস্বান্ত হতে হতো, যা একটি সামাজিক ব্যাধি হিসেবে চিহ্নিত করা হয়েছে।
Question ==>  4. "নারী কোমল হলেও দুর্বল নয়" - এই সত্য কল্যাণীর জীবনগল্পে কীভাবে প্রতিভাত হয়েছে?
কল্যাণীর জীবনগল্পে "নারী কোমল হলেও দুর্বল নয়" সত্যটি প্রতিভাত হয়েছে তার বিয়ে ভেঙে যাওয়ার পর নিজেকে শিক্ষার ব্রতে নিয়োজিত করার মাধ্যমে। যৌতুকের জন্য বিয়ে ভেঙে যাওয়ার পরেও কল্যাণী আত্মসম্মান বজায় রেখেছে এবং নিজের জীবনকে শিক্ষাদানের মাধ্যমে মানবকল্যাণে নিয়োজিত করেছে।
Question ==>  5. অনুপমের দৃষ্টান্তে মানবজীবনের চিরন্তন সত্য, আশার গুরুত্ব কীভাবে ফুটে উঠেছ

In [23]:
import csv

In [24]:
with open("question_answer.csv", mode='w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(["Question", "Answer"])  # header row

    for question in all_questions:
        print("Question ==> ", question)
        response = rag_chain.invoke({"input": question})
        print(response['answer'])
         # Write to CSV
        writer.writerow([question, response['answer']])

Question ==>  1. অনলাইন ব্যাচের সঙ্গে সম্পর্কিত যেকোনো জিজ্ঞাসার উত্তর পেতে কী করা উচিত?
উত্তর উপলব্ধ নয়।
Question ==>  2. সমাজে হঠাৎ বিত্তশালী হয়ে ওঠা নিম্নবিত্ত ব্যক্তির কারণে কী ধরনের পরিচয় সংকট দেখা দিতে পারে?
উত্তর উপলব্ধ নয়।
Question ==>  3. তৎকালীন সমাজে পণপ্রথার কুপ্রভাব সম্পর্কে আপনার কী ধারণা?
তৎকালীন সমাজে পণপ্রথার কুপ্রভাব সম্পর্কে ধারণা পাওয়া যায় যে এটি একটি সামাজিক ব্যাধি হিসেবে উপস্থিত ছিল, যার জন্য অনেকেরই আত্মসম্মানবোধহীন ও অমানবিক আচরণ করতে হত। বরপক্ষের দাবি পূরণ করতে কন্যার পিতাকে কখনো কখনো সর্বস্বান্ত হতে হত।
Question ==>  4. "নারী কোমল হলেও দুর্বল নয়" - এই সত্য কল্যাণীর জীবনগল্পে কীভাবে প্রতিভাত হয়েছে?
কল্যাণীর জীবনগল্পে "নারী কোমল হলেও দুর্বল নয়" এই সত্যটি প্রতিভাত হয়েছে তার বিয়ে না করার সিদ্ধান্তের মাধ্যমে। যৌতুকের অন্যায় আবদার প্রত্যাখ্যান করে কল্যাণী নিজের আত্মমর্যাদা ও ব্যক্তিত্বকে রক্ষা করে, যা প্রমাণ করে যে তিনি দুর্বল নন বরং আত্মনির্ভরশীল।
Question ==>  5. অনুপমের দৃষ্টান্তে মানবজীবনের চিরন্তন সত্য, আশার গুরুত্ব কীভাবে ফুটে উঠেছে?
উত্তর উপলব্ধ ন